In [2]:
import numpy as np
import quadrature4
import curve
import matplotlib.pyplot as plt

In [3]:
def figure8_generate(resolution=30, twist=0.2):
    points = []
    for i in range(resolution):
        theta = 2 * np.pi / resolution * i
        points.append(np.array([np.cos(theta), np.sin(2*theta), twist * np.sin(theta)]))
    figure8 = curve.Curve(points)
    return figure8

In [ ]:
# L2 Gradient Flow
figure8 = figure8_generate()

deltaT=0.004
Lambda=0.01
M=1000
curve = figure8
L2Energies = []

for t in range(M):
    #print(f"{t} / {M} ({t/M*100:.2f}%)")
    curve = curve - (quadrature4.dEnergy(curve, 2, 4)) * deltaT
    L2Energies.append(quadrature4.energy(curve.list_of_points))
    print(f"Progress: {t} / {M}")

In [ ]:
#H1 Gradient Flow

# Input curve, outputs matrix
def laplacianMatrix(c, identityCoeff=1):
    N = len(c)
    L = np.zeros((N, N), dtype=np.float64)

    edgeLengths = [c.edgeLength(i) for i in range(N)]
    # Fill row
    for i in range(N):
        fi = 0.5 * np.linalg.norm(c[i+1] - c[i-1])
        denominator = edgeLengths[i-1] * edgeLengths[i] * fi
        L[i][i-1] = edgeLengths[i] / denominator
        L[i][i] = - (edgeLengths[i-1] + edgeLengths[i]) / denominator
        L[i][(i+1) % N] = edgeLengths[i-1] / denominator
    
    return L + identityCoeff * np.eye(N)

figure8 = figure8_generate
alpha = 2
beta = 4
deltaT = 0.004
identityCoeff = -0.1
plotFreq = 100
H1Energies = []

# Plot
for k in range(M):
    # Compute derivative
    dE = quadrature4.dEnergy(figure8, alpha, beta).list_of_points
    # Solve Linear System
    L = laplacianMatrix(figure8, identityCoeff=identityCoeff)
    G = curve.Curve(np.linalg.solve(L, dE))
    # Evolve
    figure8 = G * deltaT + figure8
    H1Energies.append(quadrature4.energy(figure8.list_of_points))